In [1]:
import re
import random as rd

outputs = []
for i in range(5):
  # these are just large numbers to initialize 32bits
  bitstring_one = (1 << 31) | rd.getrandbits(32)
  bitstring_two = (1 << 31) | rd.getrandbits(32)
  # print("{:32b}".format(bitstring_one))
  # print("{:32b}\n".format(bitstring_two))
  curr_output = []
  for i in range(10 ** 4):
    curr_output.append((bitstring_one ^ bitstring_two) & 1)
    # print(f"{(bitstring_two >> 1) & 1} XOR {(bitstring_two >> 10) & 1}")
    bitstring_one_newbit = (bitstring_one ^ (bitstring_one >> 2) ^ (bitstring_one >> 3)) & 1
    # print(f"{(bitstring_two >> 7) & 1} XOR {(bitstring_two >> 13) & 1} XOR {(bitstring_two >> 30) & 1}")
    bitstring_two_newbit = (bitstring_two ^ (bitstring_two >> 1) ^ (bitstring_two >> 2) ^ (bitstring_two >> 3)) & 1

    # assign new bit to position 31
    bitstring_one = (bitstring_one >> 1) | (bitstring_one_newbit << 31)
    bitstring_two = (bitstring_two >> 1) | (bitstring_two_newbit << 31)

  # print("\n{:32b} ".format(bitstring_one))
  # print("{:32b} ".format(bitstring_two))
  
  outputs.append(curr_output)

output_strings = []
for output in outputs:
  output_strings.append(''.join(map(str,output)))

##### according to the NIST SP 800-22:
  - the likelyhood of a 1 should be 50% regardless of the pervious outputs. like a true conflip

  - We can mathematically calculate the longest run of a 0 or 1 and compare the expected to our lfsr output. This is an indicator of the random performance of our lfsr

  - in practice, calculate the expected length(i) repetitons of runs from 3 to 7 and compare to lfsr output.

In our case, we are looking for repetitions of 0's with length 4,5 and 6. You made this section very easy because of your *hint*  
using this formula *0.5^length+2* we can calculate the probability of 100001 when length=4.
so there should be:
  - 15.6 occurences for length=4 sequences (0.015625)
  - 7.8 occurences for length=5 sequences (0.0078125)
  - 3.9 occurences for length=6 sequences (0.00390625)


### overall distribution of 1s and 0s

In [2]:
zero_one_occurences = [] # [0 count(0s), 1 is count(1s)]*len(output_strings)
for output in output_strings:
  curr_zero_one_occurences = []
  curr_zero_one_occurences.append(output.count('0'))
  curr_zero_one_occurences.append(output.count('1'))

  zero_one_occurences.append(curr_zero_one_occurences)
zero_one_occurences

[[4992, 5008], [5085, 4915], [4957, 5043], [5055, 4945], [5005, 4995]]

0s and 1s are very even. thats great!  

### get longest sequence of 0s and 1s

In [6]:
longest_count_occurences = [] # [0 longest(0s), 1 is longest(1s)]*len(output_strings)
for output in output_strings:
  long_zero = 1
  long_one = 1
  curr_zero = 0
  curr_one = 0
  for char in output:
    if(char == '0'):
      curr_zero+=1
      if curr_zero > long_zero: long_zero=curr_zero
      curr_one = 0
    if(char == '1'):
      curr_one+=1
      if curr_one > long_one: long_one=curr_one
      curr_zero = 0

  longest_count_occurences.append([long_zero, long_one])
longest_count_occurences

[[10, 13], [13, 12], [11, 10], [12, 11], [11, 11]]

the average length or a run is 11.
the probability of just the 11 0s or 1s *0.5^11* (0.000488281)  
chance is 0.5% so it should not be happening this consistently indicating our lfsr is not very random.  
  
if we incluse the leading and trailing inverse bit it is *0.5^13* (0.00012207)
we really should not be seeing this length of sequential bits consistently.

### frequency of length 3<i<7

In [85]:
sequences = [] # [0 is i=4, 1 is i=5, 2 is i=6]*len(output_strings)
for output in output_strings:
  curr_sequences = []
  curr_sequences.append(len(re.findall("100001",output)))
  curr_sequences.append(len(re.findall("1000001",output)))
  curr_sequences.append(len(re.findall("10000001",output)))

  sequences.append(curr_sequences)
sequences

[[145, 78, 26], [152, 67, 46], [157, 62, 38], [154, 79, 39], [150, 75, 32]]

#### resuts  
our lfsr is really bad.
I keep randomly changing the xor gates to see if we get better results, however, we still get roughly these resutls:  
[[145, 78, 26], [152, 67, 46], [157, 62, 38], [154, 79, 39], [150, 75, 32]]  
~150(100001), ~70(1000001), ~35(10000001)  
it is not random and this would be insecure if it were implemented. could be completly cracked via reverse engineering.  
We could do mathematical analysis to find the best starting values and best bits to use in feedback function for the longest cycle to get a useable lfsr.